In [10]:
from selenium import webdriver

In [11]:
webdriver.Chrome()

<selenium.webdriver.chrome.webdriver.WebDriver (session="441888154efbf372bd172a7c44d882cf")>

#### 크롤링 4가지 방법 배울 것

#### 1. requests 패키지 : 2가지 방법 (API이용, html이용)
    - API이용 : API통해서 request 전송하면 json로 받고 문자열로 주고 받는다. 포맷은 json인데 string으로 받아서 json으로 바꿔주기 위해서 parsing한다. json 받은 것으로 (json데이터)
    
    - html 코드 이용 : request 던져서 html코드를 string으로 가지고 온다. css selector로 특정 엘리먼트에 있는 데이터를 가지고 오게 parsing해주는 것이 bs4(그걸로 parsing하면 css selector 오브젝트 형태로 가지고 오게 된다.) (html데이터)
    
#### 2.  selenium 
(둘다 사용 못할 때 이거 사용한다. 원래 크롤링을 위한 패키지는 아니다. 프론트엔드 페이지를 테스트하는 용도로 만들어졌다. 그래서 데이터 가지고 오고 테스트 하는 것 다 가능하다.)

    - 직접 브라우져를 띄워서 데이터를 직접 가지고 온다. css selector 이용해서 엘리먼트 가지고 온다.
    
    
차이점 : 정적인 페이지(html페이지 보여주고 끝), 동적인 웹페이지(html 페이지 보여주는 상태에서 api를 이용해서 서버에 js 코드 가져오는 요청해서 데이터 받아와서 추가로 더해주는 경우)

- 정적인 웹페이지 크롤링 : request - html
- 동적인 웹페이지 크롤링 : selenium


모든 방법이 크롤링 가능 할 때 request API 이용 : html 코드 이용시 필요없는 엘리먼트 이름들 많아서 API 이용하는 것이 속도 더 빠르다.
html, selenium 가능 할 시 : html은 그냥 데이터 가지고 오지만 selenium 은 웹 브라우져 띄우기 때문에 많은 페이지 크롤링 시 속도가 느리다.

API이용 > HTML > Selenium


#### 3. scrapy : 웹크롤링을 위해서 만들어진 파이썬 패키지

    - 다른 방법으로 안되는 것 가능한 기능들이 많다. robots.txt 정책 설정 할 수 있다.
    - 동적인 웹페이지 크롤링 위해서는 selenium 가져와서 쓴다.
    - xpath selector : scrapy에서 많이 사용한다.
    
chrome driver의 headless설정(화면을 띄우지 않은 상태에서 크롤링 하는 방법)
phantomjs : 더이상 지원하지 않음 js라이브러리이지만
xvfb : virture window를 보이지 않지만 띄워주면서 

In [12]:
import requests

In [13]:
from bs4 import BeautifulSoup

In [14]:
import forecastio

- api json을 사용 : 네이버 주식 데이터 크롤링
- api json을 사용, forecastio : dark sky api로 날씨 데이터를 수집 (dark sky 서비스를 가입을 해서)
- bs4 : 네이버 실시간 키워드, 다음 실시간 키워드 크롤링
- web file url로 파일을 다운로드 받는 방법

#### 네이버 주식데이터 가져오기
- 대부분 웹페이지 모바일 페이지 가지고 있는데 모바일이 더 쉽게 되어 있다.
- xhr : 네트워크 트래픽을 보여준다.

- 200 성공 400 실패 500 
- get / post 방식
- request header : request 할 때의 정보 
    - cookie 로그인 정보 브라우저 숨겨진 쿠키 정보들도 같이 보내고 있다. 그전에 어떤 페이지 갔었는지 알 수 있음 
    - referer : 이전에 어떤 페이지 였는지 알려준다.
    
- response 탭 : 서버에서 클라이언트로 전송하는 정보 볼 수 있다. response도 그냥 문자열로 받는다.

- preview 탭: 리스폰스 받은 것 보기 좋게 정리해 준 것, 

pageSize=20 (한번에 20개)&page=9 (지금 9 번째 페이지)

In [15]:
import pandas as pd

In [21]:
# 페이지 사이즈와 페이지 값을 받자
# chrome 개발자 도구에서 network > xhr > 새 페이지로 열기

def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize="+str(pageSize)+"&page="+ str(page)

In [22]:
from pandas.io.json import json_normalize

In [26]:
# 데이터를 가져와서 판다스의 데이터 프레임으로 만드는 함수

def get_data(url):
    response = requests.get(url) # get으로 데이터를 요청하는 것, 크롬 개발자 도구에서 get post 확인 후 함수 설정 해준다.
    json_info = response.json() #json포맷의 object로 파싱한다.
    #print(json_info) # response 객체 안에 text 변수에 받은 것을 
    companys = json_info["result"]["itemList"] #종목별로 가지고 온다. 리스트로 결과가 나온다.
    
#     df = pd.DataFrame(columns=["종목","시세","전일비","등락율","시가총액","거래량"]) # row를 하나씩 넣는 방법 사용
    
#     for company in companys:
#         df.loc[len(df)] = {
#             "종목":company["nm"],
#             "시세":company["nv"],
#             "전일비":company["cv"],
#             "등락율":company["cr"],
#             "시가총액":company["mks"],
#             "거래량":company["aq"],
#         }
    
    return json_normalize(companys)

In [27]:
url = make_url(1500, 1)
get_data(url)

,aa,aq,cd,cr,cv,mks,mt,nm,nv,pcv,rf
0,533872,11309331,005930,0.00,0,3017083,0,삼성전자,47000,47000,3
1,362216,4165547,000660,4.40,3700,639186,0,SK하이닉스,87800,84100,2
2,276793,933448,068270,3.30,9500,372426,0,셀트리온,297000,287500,2
3,40335,1082753,005935,0.27,100,335246,0,삼성전자우,37100,37000,2
4,70766,206088,005490,0.74,2500,298615,0,POSCO,342500,340000,2
5,73107,562722,005380,-1.52,-2000,285258,0,현대차,129500,131500,5
6,164679,399336,207940,10.24,39000,277893,0,삼성바이오로직스,420000,381000,2
7,92882,262351,051910,1.14,4000,249544,0,LG화학,353500,349500,2
8,49207,33455,051900,4.97,70000,230993,0,LG생활건강,1479000,1409000,2
9,59782,1080469,105560,0.00,0,230379,0,KB금융,55100,55100,3


- - -
#### 날씨 정보 데이터 dark sky api
- 위도와 경도를 입력하면 날씨 정보를 보내주는 api
- https://api.darksky.net/

In [50]:
FORECAST_TOKEN = "d42b9b5217897e99db3140753294b90c"

In [74]:
# 직접 가지고 오는 법
    # lastAPI : data를 path에 쓰는 방법
def forecast(lat, lng):
    url = "https://api.darksky.net/forecast/{}/{},{}".format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url) # 요청하기
    json_info = response.json() # json으로 파싱하기
    return json_info["timezone"], json_info['hourly']['summary']

In [75]:
lat = 37.512
lng = 126.954
forecast(lat, lng)

('Asia/Seoul', 'Mostly cloudy throughout the day.')

In [76]:
# 이미 누가 패키지 만들어 놨다.

def forecast2(lat, lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng)
    byHourly = forecast.hourly()
    return byHourly.summary

In [77]:
lat = 37.512
lng = 126.954
forecast2(lat, lng)

'Mostly cloudy throughout the day.'

- - -
### bs4 실습
#### 네이버 키워드 랭킹 데이터 크롤링
#### 다음 키워드 랭킹 데이터 크롤링

In [28]:
def naver():
    df = pd.DataFrame(columns=["rank","keyword"]) # 데이터 프레임으로 만들 거라서 df만든다.
    
    response = requests.get("https://www.naver.com/") # request 로 네이버 현재 페이지의 html코드 가지고 오기, 깨질 경우from_encoding 설정해줘야한다.
    dom = BeautifulSoup(response.content, "html.parser") #content 한글이나 \ 이런거 다 그대로 보여준다. html.parser한 결과가 dom이라서 dom 이라고 적어 둔것 dom : html 구조
    dom.select # select를 한다. select: 여러개의 html element를 셀렉팅 할때 사용, 결과로 리스트 데이터를 리턴/ select_one : 하나의 html엘리먼트를 셀렉팅 할 때 사용, 결과로 문자열 데이터를 리턴
    keywords = dom.select(".ah_roll_area > .ah_l > .ah_item") # css selector 여기에 적는다. 갯수가 안맞을 경우 다른 곳에도 같은 클래스 이름을 사용하고 있어서 상위 클래스 이름을 추가로 적어서 정확한 css selector 작동하도록 만든다.
    
    for keyword in keywords: #키워드 하나하나 element object다
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ah_r").text, 
            "keyword": keyword.select_one(".ah_k").text,
        }
                # 해당 엘리먼트 안의 text를 뽑고 싶을 때 .text 적는다.
    return df


# 한시간에 한번씩 수집 해야 할때



# 클래스가 하나 둘 띄워쓰기로 두개 있는 경우 있는데 그 경우에는 붙여서 .ah_roll_area.두번째클래스 이런식으로 적으면 된다.
# 선택하고 싶은 css selector 좌클릭하면 카피가능

In [123]:
def daum():
    df = pd.DataFrame(columns=["rank","keyword"])
    
    response = requests.get("https://www.daum.net/")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".list_hotissue issue_row")
    print(len(keywords))
    
    return df
    
    

In [152]:
def daum():
    df = pd.DataFrame()
    rank = []
    keyword = []
    
    response = requests.get("https://www.daum.net/")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".realtime_part > .list_hotissue.issue_row > li") #여러개 엘리먼트 셀렉트 가능
    #print(len(keywords))
    rank = [keyword.select_one(".ir_wa").text[:-1] for keyword in keywords]
    keywords = [keyword.select_one(".link_issue").text for keyword in keywords]
    # print(rank)
    df["rank"] = rank
    df["keyword"] = keywords
    
    return df

In [153]:
#mArticle > div.cmain_tmp > div.section_media > div.hotissue_builtin > div.realtime_part > ol > li.next > div

In [154]:
daum_df = daum()

In [29]:
naver_df = naver()

In [157]:
daum_df["keyword"], naver_df["keyword"]

(0       민갑룡
 1      경찰청장
 2     국가장학금
 3    한국장학재단
 4       조현배
 5      블랙핑크
 6       심영은
 7       안희정
 8       김무성
 9       이찬오
 Name: keyword, dtype: object, 0          이집트 우루과이
 1              뮤직뱅크
 2             포항 약국
 3              블랙핑크
 4              광주날씨
 5                쌈디
 6            모로코 이란
 7     러시아 사우디 하이라이트
 8                살라
 9               민갑룡
 10              강혁민
 11              박기량
 12              심영은
 13              ksl
 14              최원명
 15             경찰청장
 16         포르투갈 스페인
 17            국가장학금
 18        한국예술인복지재단
 19           프로듀스48
 Name: keyword, dtype: object)

In [161]:
# 공통 키워드 찾기

result = [
    keyword
    for keyword in naver_df["keyword"]
    if daum_df["keyword"].str.contains(keyword).any()
]

result

# set으로 교집합 찾아도 된다.

['블랙핑크', '민갑룡', '심영은', '경찰청장', '국가장학금']

In [97]:
import time
ls = []
for idx in range(3):
    print(idx)
    ls.append(naver()) # ls에 데이터 프레임이 하나하나 들어간다.
    time.sleep(5) # 기본 초단위이고, 5시간 원하면 60*60*5 이런식으로 적는게 가독성 좋다. 5초마다 어팬드 하고 3번!

0
1
2


In [101]:
ls[1]

,rank,keyword
0,1,조현배
1,2,이집트 우루과이
2,3,쌈디
3,4,민갑룡
4,5,살라
5,6,포항 약국
6,7,심영은
7,8,경찰청장
8,9,러시아 사우디 하이라이트
9,10,장윤정 엄마


#### file download : requests
- file의 url을 받아서 해당 url의 file을 다운로드 하는 방법
- 유투브 영상 다운받아보자

In [162]:
len("https://r8---sn-3u-bh2l7.googlevideo.com/videoplayback?pl=17&dur=68.963&signature=7F7A515A2D350FFDACAA976C894BC1DF9FBF2FA7.565EC4FA8D07AE98B83969733FD61C21E580A640&ratebypass=yes&source=youtube&requiressl=yes&c=WEB&id=o-AJFVkbSJ2hcn_3M2BfV9Qd2ibWaLo-fUtwn4UHho_Cks&fvip=5&mime=video%2Fmp4&key=cms1&expire=1529059002&lmt=1529027498525595&ipbits=0&ei=WkIjW4PkFIaA8wTNko3IAg&itag=22&sparams=dur,ei,expire,id,ip,ipbits,ipbypass,itag,lmt,mime,mip,mm,mn,ms,mv,pcm2cms,pl,ratebypass,requiressl,source&ip=172.241.147.96&video_id=dar_brj8zdw&title=Mac+%E2%80%94+Behind+the+Mac+%E2%80%94+Apple&redirect_counter=1&rm=sn-aigesy76&req_id=99123b8eef92a3ee&cms_redirect=yes&ipbypass=yes&mip=220.117.71.145&mm=31&mn=sn-3u-bh2l7&ms=au&mt=1529051565&mv=m&pcm2cms=yes")

748

In [171]:
# 긴 url 줄이는 법 bit.ly, 현재 디렉토리에 url연결해서 title으로 다운로드 한다.

download_link = "https://bit.ly/2ydrpsz"
title = "video.mp4"

In [172]:
def download(title, download_link):
    response = requests.get(download_link, stream=True) # 파일 다운 받는 거라서 stream  true 해줌
    file_size = 0
    with open(title, 'wb') as f: # 파일 열어준다.
        for chunk in response.iter_content(chunk_size=1024): # byte # response를 한번에 저장하는게 아니라 chunk이용해서 1024바이트로 잘라서 저장한다.
            if chunk:
                file_size += 1024 # with 로 하면 close 하지 않아도 된다.
                f.write(chunk) #ftp에서도 chunk 사용된다. 큰 파일 한꺼번에 전송하는데 문제가 생겨서 특정 영역의 데이터가 깨졌을 때 처음 그파일 그대로 다시 보낸다. 그럼 효율이 떨어져서 잘라서 보내는 이유가  중간에 몇개깨져도 조금만 다시 보내면 되니까 잘라서 저장하는 이유
    return file_size

# 파일 단위 byte 단위로 기록되면 지저분하니까 mb로 바꾼다.

In [175]:
file_size = download(title, download_link)

In [176]:
str(round(file_size/1024/1024,2)) + "Mb"

'0.0Mb'

- bit 0.1
- byte = 8bit
- 1KB = 1024byte 1024씩 올라간다. 2**10
- 1MB = 1024kb

In [ ]:
# 1gbit byte